In [ ]:
import geopandas as gpd
import osmnx as ox
from geojson import LineString
from matplotlib import pyplot as plt
from shapely.affinity import translate

from config import CPH_AMAGER_DANGER_ZONE
from data_loader import load_json_file_to_str
from data_loader.danger_zones import load_danger_zone_from_str
from data_loader.osm import load_osm
from routes.fastest_path import FastestPath
from routes.route_algo import RouteAlgo
from routes.shortest_path import ShortestPath

In [ ]:
# Load the OSM data for Copenhagen
graph = load_osm("copenhagen.graphml")
nodes, edges = ox.graph_to_gdfs(graph)

In [ ]:
# Compute paths to safety
origin = 16965546
danger_zones = load_danger_zone_from_str(
    load_json_file_to_str(CPH_AMAGER_DANGER_ZONE), "EPSG:4326"
)


def pathfind(algorithm: RouteAlgo) -> gpd.GeoSeries:
    path = algorithm.route_to_safety([origin], danger_zones, graph)[origin][0]
    path_coordinates = [
        (graph.nodes[node]["x"], graph.nodes[node]["y"]) for node in path
    ]
    return gpd.GeoSeries([LineString(path_coordinates)], crs=edges.crs)


fastest_line = pathfind(FastestPath())
shortest_line = pathfind(ShortestPath())

In [ ]:
# Project the GeoDataFrames to UTM zone 32N (EPSG:32632)
epsg = 32632
nodes = nodes.to_crs(epsg=epsg)
edges = edges.to_crs(epsg=epsg)

# Offset the lines to avoid overlap in the plot
path_gdfs = [
    fastest_line.to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=0)),
    shortest_line.to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=30)),
]

In [ ]:
# Plot the graph with the paths overlayed. Zoomed in on the paths with a buffer around them.
buffer = 1_000

fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='gray')
# nodes.plot(ax=ax, markersize=1, color='black')
for gdf, color in zip(path_gdfs, ['red', 'blue']):
    gdf.plot(ax=ax, linewidth=3, color=color)

minx, miny, maxx, maxy = gpd.GeoSeries([g[0] for g in path_gdfs], crs=edges.crs).total_bounds
ax.set_xlim(minx - buffer, maxx + buffer)
ax.set_ylim(miny - buffer, maxy + buffer)

# ax.set_title("Graph with Path Overlay on Amager", fontsize=16)
ax.set_aspect('equal')
ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.show()